In [430]:
%esptool erase



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f61286e6250, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Executing:
  esptool.py --port /dev/ttyUSB1 erase_flash

esptool.py v2.8
Serial port /dev/ttyUSB1
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: 68:c6:3a:99:41:c2
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 8.3s
Hard resetting via RTS pin...


In [431]:
%esptool esp8266 bin/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB1 --baud 460800 write_flash --flash_size=detect -fm dio 0 bin/firmware-combined.bin

esptool.py v2.8
Serial port /dev/ttyUSB1
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: 68:c6:3a:99:41:c2
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 624400 bytes to 406630...
Wrote 624400 bytes (406630 compressed) at 0x00000000 in 9.4 seconds (effective 531.7 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [433]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [434]:
# Setting up the webrepl hotspot with password=wpass
%serialconnect --raw
%readbytes 

# Print out the hotspot name (while we are here)
%writebytes "import network\r\n"
%readbytes
%writebytes "\r\n"
%readbytes
%writebytes "'MicroPython-{:02x}{:02x}{:02x}'.format(*network.WLAN().config('mac')[-3:])\r\n"
%comment "This is the ESP8266's hotspot name"
%readbytes
%writebytes "\r\n"
%readbytes

# soft reset so import works
%writebytes "\x04"
%readbytes
%writebytes "import webrepl_setup\r\n"
%readbytes 
%writebytes "E\r\n"
%readbytes

# uncomment if second time (with additional question of changing password)
#%writebytes "y\r\n"
#%readbytes

# set the password
%writebytes "wpass\r\n"
%readbytes
%writebytes "wpass\r\n"
%readbytes

#%writebytes "y\r\n"
#%readbytes

%writebytes "\x04"
%readbytes
%disconnect --raw

Connecting to --port=/dev/ttyUSB2 --baud=115200 
import network
>>> 
>>> This is the ESP8266's hotspot name'MicroPython-{:02x}{:02x}{:02x}'.format(*network.WLAN().config('mac')[-3:])
'MicroPython-9941c2'
>>> 
>>> 
MPY: soft reboot
MicroPython v1.11-623-gd61e7a6d8-dirty on 2019-12-05; ESP module with ESP8266
Type "help()" for more information.
>>> import webrepl_setup
WebREPL daemon auto-start status: disabled

Would you like to (E)nable or (D)isable it running on boot?
(Empty line to quit)
> E
To enable WebREPL, you must set password for it
New password (4-9 chars): wpass
Confirm password: wpass

Closing serial Serial<id=0x7f61286f8610, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [607]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [604]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   sensorcity.io
pinled       2
boardname    abdushop9
pinbitbashin 13

Sent 6 lines (131 bytes) to config.txt.


In [611]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [608]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 31 lines (1106 bytes) to stdmqttas.py.
Sent 32 lines (999 bytes) to utils.py.


In [612]:
%sendtofile main.py

import time, itertools, network
network.WLAN().active(0)

from machine import Pin, time_pulse_us
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, pinled
from mqtt_as import MQTTClient

shortmacaddress ="{:02x}{:02x}{:02x}".format(*network.WLAN().config("mac")[-3:])
print("*** connected shortmac", shortmacaddress)
boardname = fconfig["boardname"]
topicstatus = "ab/%s/%s/status"%(boardname, shortmacaddress)
topicserial = "ab/%s/%s/serial"%(boardname, shortmacaddress)

pinbitbashin = Pin(int(fconfig["pinbitbashin"]), Pin.IN)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topicstatus, shortmacaddress, retain=True)
    await client.publish(topicstatus, ipnumber, retain=True)

async def heartbeattask(client):
    for i in itertools.count():
        payload = "beat %d"%i
        print(payload)
        if client.isconnected():
            await client.publish(topicstatus, payload, retain=True)
        await asyncio.sleep_ms(3000)

async def receivebitbashtask(client):
    dat = bytearray(100)
    while True:
        while pinbitbashin.value() == 1:
            await asyncio.sleep_ms(5)
        x = (time_pulse_us(pinbitbashin, 1, 50000)+500)//1000
        if x == 3:
            for i in range(100):
                x = (time_pulse_us(pinbitbashin, 1, 5000)+500)//1000
                if x == 1:
                    dat[i] = ord('0')
                elif x == 3:
                    dat[i] = ord('1')
                elif i != 0:
                    print("short", i)
            else:
                print(dat)
                if client.isconnected():
                    await client.publish(topicserial, dat)
   
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(heartbeattask(client))
aloop.create_task(mqttconnecttask(client))
aloop.create_task(receivebitbashtask(client))
aloop.run_forever()




Sent 61 lines (2053 bytes) to main.py.


In [442]:
import network
w = network.WLAN()
w.active(1)
w.connect("DoESLiverpool", "decafbad00")
import socket
print(socket.getaddrinfo("sensorcity.io", 80))

Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
AttributeError: 'module' object has no attribute 'getaddr'


In [446]:
print(socket.getaddrinfo("sensorcity.io", 80))

[(2, 1, 0, '', ('3.15.149.233', 80))]


In [483]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [534]:
from machine import Pin
pinbitbashin = Pin(13, Pin.IN)
def readpulses():
    while pinbitbashin.value():
        pass
    x1 = time_pulse_us(pinbitbashin, 1, 500000)
    x = (x1+500)//1000
    if x != 3:
        return [x1]
    for i in range(100):
        x = (time_pulse_us(pinbitbashin, 1, 5000)+500)//1000
        if x == 1:
            dat[i] = ord('0')
        elif x == 3:
            dat[i] = ord('1')
        elif i != 0:
            return i, dat[:i]
    return dat
print(readpulses())


[-2]


In [49]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [304]:
from machine import Pin, time_pulse_us
pinbitbashin = Pin(13, Pin.IN)

dat = bytearray(100)
def readpulses():
    if pinbitbashin.value() != 0:
        return ""
    x = (time_pulse_us(pinbitbashin, 1, 50000)+500)//1000
    if x != 3:
        return ""
    print("got3")
    for i in range(100):
        x = (time_pulse_us(pinbitbashin, 1, 5000)+500)//1000
        if x == 1:
            dat[i] = ord('0')
        elif x == 3:
            dat[i] = ord('1')
        elif i != 0:
            return ""
    return dat


In [320]:
while pinbitbashin.value() == 1:
    pass
t = readpulses()
if t:
    print(t)


got3
bytearray(b'1101111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111')


In [223]:
x = (time_pulse_us(pinbitbashin, 0, 500000)+500)//1000
print(x)

50


In [599]:
print(pinbitbashin.value())

0
